## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-23-45-33 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
1,2025-07-12-23-40-23 +0000,bbc,BBC gains rare access to the Congolese mine po...,https://www.bbc.com/news/articles/cyvj986l615o
2,2025-07-12-23-38-06 +0000,nypost,Sonic scientists create fiery show in Brooklyn...,https://nypost.com/2025/07/12/us-news/fire-spe...
3,2025-07-12-23-22-07 +0000,nypost,Bringing down the dog house: cute pets gather ...,https://nypost.com/2025/07/12/us-news/stars-tu...
4,2025-07-12-23-13-58 +0000,bbc,North Korea reaffirms support for Russia's war...,https://www.bbc.com/news/articles/czeyxrng6gro


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
64,trump,10
1,india,5
0,air,5
129,president,5
3,crash,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2025-07-12-21-17-00 +0000,wsj,President Trump traveled to the site of the Te...,https://www.wsj.com/politics/policy/trump-texa...,39
36,2025-07-12-12-00-00 +0000,wsj,Nearly six months into President Trump’s secon...,https://www.wsj.com/politics/policy/federal-la...,34
30,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,32
16,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...,27
35,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...,27


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,39,2025-07-12-21-17-00 +0000,wsj,President Trump traveled to the site of the Te...,https://www.wsj.com/politics/policy/trump-texa...
0,25,2025-07-12-23-45-33 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
54,20,2025-07-12-00-00-00 +0000,wsj,The U.S.’s dramatic research and funding cuts ...,https://www.wsj.com/politics/policy/us-brain-d...
31,19,2025-07-12-16-58-40 +0000,nypost,College student attacked over NYC parking spot...,https://nypost.com/2025/07/12/us-news/college-...
16,17,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...
36,17,2025-07-12-12-00-00 +0000,wsj,Nearly six months into President Trump’s secon...,https://www.wsj.com/politics/policy/federal-la...
30,17,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
15,16,2025-07-12-21-11-05 +0000,bbc,The mushroom killer was obsessed with true cri...,https://www.bbc.com/news/articles/c0m8glx2zleo
34,15,2025-07-12-15-00-00 +0000,wsj,Pardoned Jan. 6 defendants are the new draw at...,https://www.wsj.com/politics/jan-6-rioters-are...
35,15,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
